In [5]:
import cv2
from ultralytics import YOLO
import numpy as np
from collections import deque
import time
import math

# ==================== SETTINGS & CONSTANTS ====================
DETECTION_CONFIDENCE = 0.5

# Base Angles
BASE_ANGLE_UP = 160    # Standing straight
BASE_ANGLE_DOWN = 110  # Deep squat (lower is harder)

# Thresholds for Form Correction
MAX_TORSO_LEAN = 35    # Max degrees torso can lean forward
MAX_HIP_DIFF = 50      # Max pixel difference between hips
KNEE_OVER_TOE_THRESHOLD = 40  # Pixels knee can go past ankle

# Tolerance Settings (Dynamic Adjustment)
tolerance_level = 0
ANGLE_UP_THRESHOLD = BASE_ANGLE_UP
ANGLE_DOWN_THRESHOLD = BASE_ANGLE_DOWN

# YOLO Keypoint Indices
# 5,6: Shoulders | 11,12: Hips | 13,14: Knees | 15,16: Ankles
KP_MAP = {
    "LEFT":  {"shoulder": 5, "hip": 11, "knee": 13, "ankle": 15},
    "RIGHT": {"shoulder": 6, "hip": 12, "knee": 14, "ankle": 16}
}

# ==================== STATE VARIABLES ====================
current_phase = "UP"
total_reps = 0
total_score = 0
rep_error_flag = 0
current_errors = []

# Leg Management
front_leg = "RIGHT"
is_leg_locked = False

# Rep Tracking
min_angle_this_rep = 999  # Track lowest angle during rep
last_rep_time = 0

# Buffers & Stability
angle_history = deque(maxlen=5)
missing_frames_count = 0
MAX_MISSING_FRAMES = 30

# UI Buffer (Prevents flickering)
last_valid_ui_data = {
    "reps": 0, "score": 0, "phase": "UP", 
    "leg": "RIGHT", "angle": 180, "errors": []
}

# Load Model
print("=" * 50)
print("   BULGARIAN SPLIT SQUAT - AI TRAINER")
print("=" * 50)
print("Loading YOLOv11 Model...")
model = YOLO('yolo11n-pose.pt')
print("Model loaded successfully!")
print("=" * 50)

# ==================== HELPER FUNCTIONS ====================

def update_tolerance(change):
    """
    Adjusts exercise difficulty dynamically.
    Positive = easier, Negative = harder
    """
    global tolerance_level, ANGLE_UP_THRESHOLD, ANGLE_DOWN_THRESHOLD
    
    tolerance_level = max(-10, min(20, tolerance_level + change))
    ANGLE_UP_THRESHOLD = BASE_ANGLE_UP - tolerance_level
    ANGLE_DOWN_THRESHOLD = BASE_ANGLE_DOWN + tolerance_level
    
    print(f"Tolerance: {tolerance_level:+d} | UP: {ANGLE_UP_THRESHOLD}° | DOWN: {ANGLE_DOWN_THRESHOLD}°")


def calc_angle_3pts(a, b, c):
    """
    Calculates angle at point B between vectors BA and BC.
    
    Args:
        a, b, c: Points as [x, y] arrays
    
    Returns:
        Angle in degrees (0-180)
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.degrees(np.arccos(np.clip(cosine, -1, 1)))
    
    return angle


def calc_torso_angle(hip, shoulder):
    """
    Calculates torso lean angle relative to vertical.
    
    In OpenCV coordinate system:
    - Y increases downward
    - 0 degrees = perfectly upright
    - Positive = leaning forward
    
    Args:
        hip: [x, y] of hip point
        shoulder: [x, y] of shoulder point
    
    Returns:
        Angle in degrees (0 = straight up)
    """
    dx = shoulder[0] - hip[0]
    dy = shoulder[1] - hip[1]  # Negative because shoulder is above hip
    
    # atan2(-dy, dx) gives angle from vertical
    # We use -dy because in image coords, up is negative Y
    angle = abs(math.degrees(math.atan2(dx, -dy)))
    
    return angle


def detect_front_leg(kp):
    """
    Determines which leg is in front based on ankle Y position.
    
    In Bulgarian Split Squat:
    - Front foot is flat on floor (higher Y value in OpenCV)
    - Back foot is elevated on bench (lower Y value)
    
    Args:
        kp: Keypoints array from YOLO
    
    Returns:
        "RIGHT" or "LEFT"
    """
    right_ankle_y = kp[16][1]
    left_ankle_y = kp[15][1]
    
    return "RIGHT" if right_ankle_y > left_ankle_y else "LEFT"


def check_knee_over_toes(kp, leg, knee_angle):
    """
    Checks if knee is extending too far past the toes.
    
    This is a common form mistake that puts stress on the knee.
    Only checked when knee is bent (angle < 130).
    
    Args:
        kp: Keypoints array
        leg: "LEFT" or "RIGHT"
        knee_angle: Current knee angle
    
    Returns:
        True if knee is over toes (bad form)
    """
    if knee_angle >= 130:
        return False
    
    idx = KP_MAP[leg]
    knee_x = kp[idx["knee"]][0]
    ankle_x = kp[idx["ankle"]][0]
    
    # Direction depends on which leg
    if leg == "RIGHT":
        # Right leg: knee going right is forward
        return knee_x > ankle_x + KNEE_OVER_TOE_THRESHOLD
    else:
        # Left leg: knee going left is forward
        return knee_x < ankle_x - KNEE_OVER_TOE_THRESHOLD


def check_hip_balance(kp):
    """
    Checks if hips are level (not tilted to one side).
    
    Uneven hips indicate poor balance or compensation.
    
    Args:
        kp: Keypoints array
    
    Returns:
        True if hips are unbalanced (bad form)
    """
    left_hip_y = kp[11][1]
    right_hip_y = kp[12][1]
    
    return abs(left_hip_y - right_hip_y) > MAX_HIP_DIFF


def check_form(kp, leg, knee_angle, phase, min_angle):
    """
    Comprehensive form checker for Bulgarian Split Squat.
    
    Checks:
    1. Torso lean - Is the chest staying upright?
    2. Knee over toes - Is knee tracking properly?
    3. Hip balance - Are hips level?
    4. Depth - Did the user go deep enough?
    
    Args:
        kp: Keypoints array
        leg: Active leg ("LEFT" or "RIGHT")
        knee_angle: Current smoothed knee angle
        phase: Current phase ("UP" or "DOWN")
        min_angle: Minimum angle reached during this rep
    
    Returns:
        List of error messages (empty if form is good)
    """
    errors = []
    idx = KP_MAP[leg]
    
    # Get relevant points
    shoulder = kp[idx["shoulder"]][:2]
    hip = kp[idx["hip"]][:2]
    
    # --- 1. TORSO LEAN CHECK ---
    # Only check during DOWN phase when it matters most
    if phase == "DOWN":
        torso_angle = calc_torso_angle(hip, shoulder)
        
        if torso_angle > MAX_TORSO_LEAN:
            errors.append("KEEP CHEST UP")
    
    # --- 2. KNEE OVER TOES CHECK ---
    # Check when knee is significantly bent
    # if phase == "DOWN" and check_knee_over_toes(kp, leg, knee_angle):
    #     errors.append("KNEE OVER TOES")
    
    # --- 3. HIP BALANCE CHECK ---
    # Check throughout the movement
    if check_hip_balance(kp):
        errors.append("LEVEL YOUR HIPS")
    
    # --- 4. DEPTH CHECK ---
    # Only check when coming back up from DOWN phase
    # If minimum angle was too high, user didn't go deep enough
    if phase == "DOWN" and knee_angle > 140:
        # User is coming back up
        if min_angle > ANGLE_DOWN_THRESHOLD + 15:
            errors.append("GO DEEPER")
    
    return errors


# ==================== UI DRAWING FUNCTIONS ====================

def draw_ui(img, data, fps):
    """
    Draws the complete UI overlay.
    
    Includes:
    - Left sidebar with stats
    - Phase indicator
    - Depth bar
    - Error/success message
    - FPS counter
    """
    h, w = img.shape[:2]
    
    # --- SIDEBAR BACKGROUND ---
    cv2.rectangle(img, (0, 0), (280, h), (18, 18, 24), -1)
    cv2.line(img, (280, 0), (280, h), (50, 50, 60), 2)

    # --- REP COUNTER ---
    cv2.putText(img, "REPS", (30, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)
    cv2.putText(img, str(data["reps"]), (30, 130), 
                cv2.FONT_HERSHEY_DUPLEX, 2.5, (255, 255, 255), 3)

    # --- PHASE INDICATOR ---
    if data["phase"] == "DOWN":
        phase_color = (0, 255, 120)  # Green
        phase_text = "DOWN"
    else:
        phase_color = (0, 140, 255)  # Orange
        phase_text = "UP"
        
    cv2.rectangle(img, (30, 160), (250, 210), phase_color, -1)
    cv2.putText(img, phase_text, (100, 197), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # --- ACTIVE LEG ---
    leg_color = (0, 255, 255) if data["leg"] == "RIGHT" else (255, 0, 255)
    cv2.putText(img, f"LEG: {data['leg']}", (30, 260), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, leg_color, 2)
    
    # --- TOLERANCE LEVEL ---
    tol = tolerance_level
    if tol > 0:
        tol_color = (0, 255, 0)   # Green = easier
        tol_text = f"EASY +{tol}"
    elif tol < 0:
        tol_color = (0, 0, 255)   # Red = harder
        tol_text = f"HARD {tol}"
    else:
        tol_color = (200, 200, 200)
        tol_text = "NORMAL"
    
    cv2.putText(img, tol_text, (30, 300), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, tol_color, 1)
    # --- TOLERANCE GUIDE ---
    cv2.putText(img, "[+/-] Adjust", (30, 325), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (100, 100, 100), 1)

    # --- DEPTH BAR ---
    bar_x = 230
    bar_y = 350
    bar_w = 30
    bar_h = 200
    
    angle = data["angle"]
    norm_angle = np.clip(angle, 90, 170)
    progress = 1 - ((norm_angle - 90) / 80)  # 0 to 1
    
    # Background
    cv2.rectangle(img, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), 
                  (40, 40, 40), -1)
    
    # Fill based on depth
    fill_h = int(progress * bar_h)
    
    if progress > 0.8:
        fill_color = (0, 255, 0)      # Green - excellent
    elif progress > 0.5:
        fill_color = (0, 255, 255)    # Yellow - good
    else:
        fill_color = (0, 0, 255)      # Red - needs more depth
    
    cv2.rectangle(img, (bar_x, bar_y + bar_h - fill_h), 
                  (bar_x + bar_w, bar_y + bar_h), fill_color, -1)
    
    # Border
    cv2.rectangle(img, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), 
                  (100, 100, 100), 1)
    
    # Angle text
    cv2.putText(img, f"{int(angle)}", (150, bar_y + bar_h), 
                cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 2)
    
    # Label
    cv2.putText(img, "DEPTH", (30, bar_y + 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 150, 150), 1)

    # --- SCORE ---
    avg_score = data["score"] // data["reps"] if data["reps"] > 0 else 0
    
    score_color = (0, 255, 0) if avg_score >= 90 else \
                  (0, 255, 255) if avg_score >= 70 else (0, 0, 255)
    
    cv2.putText(img, f"AVG SCORE: {avg_score}", (30, h - 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, score_color, 2)

    # --- ERROR / SUCCESS MESSAGE ---
    msg_x = 300
    msg_y = 20
    msg_w = 400
    msg_h = 60
    
    if data["errors"]:
        # Error state - red box
        cv2.rectangle(img, (msg_x, msg_y), (msg_x + msg_w, msg_y + msg_h), 
                      (0, 0, 180), -1)
        cv2.rectangle(img, (msg_x, msg_y), (msg_x + msg_w, msg_y + msg_h), 
                      (0, 0, 255), 2)
        
        # Warning icon
        cv2.putText(img, "!", (msg_x + 20, msg_y + 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
        
        # Error text
        cv2.putText(img, data["errors"][0], (msg_x + 60, msg_y + 42), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    else:
        # Success state - green box
        cv2.rectangle(img, (msg_x, msg_y), (msg_x + 250, msg_y + msg_h), 
                      (0, 100, 0), -1)
        cv2.rectangle(img, (msg_x, msg_y), (msg_x + 250, msg_y + msg_h), 
                      (0, 200, 0), 2)
        
        cv2.putText(img, "PERFECT FORM", (msg_x + 20, msg_y + 42), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200, 255, 200), 2)

    # --- FPS ---
    cv2.putText(img, f"FPS: {int(fps)}", (w - 120, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
    
    # --- CONTROLS GUIDE ---
    cv2.putText(img, "[R] Reset  [Q] Quit  [+/-] Difficulty", 
                (w - 380, h - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)


def draw_skeleton(img, kp, leg, angle):
    """
    Draws the active leg skeleton with angle visualization.
    
    Features:
    - Color-coded based on depth
    - Angle text on knee
    - Joint markers
    """
    idx = KP_MAP[leg]
    
    # Convert points to integers
    hip = tuple(map(int, kp[idx["hip"]][:2]))
    knee = tuple(map(int, kp[idx["knee"]][:2]))
    ankle = tuple(map(int, kp[idx["ankle"]][:2]))
    
    # Color based on angle
    if angle < ANGLE_DOWN_THRESHOLD:
        color = (0, 255, 0)       # Green - deep squat
    elif angle > ANGLE_UP_THRESHOLD:
        color = (0, 140, 255)     # Orange - standing
    else:
        color = (0, 255, 255)     # Yellow - mid range
    
    # --- DRAW LINES ---
    # Outline (black)
    cv2.line(img, hip, knee, (0, 0, 0), 7)
    cv2.line(img, knee, ankle, (0, 0, 0), 7)
    
    # Colored line
    cv2.line(img, hip, knee, color, 4)
    cv2.line(img, knee, ankle, color, 4)
    
    # --- DRAW JOINTS ---
    # Hip
    cv2.circle(img, hip, 10, (255, 0, 255), -1)
    cv2.circle(img, hip, 10, (255, 255, 255), 2)
    
    # Ankle
    cv2.circle(img, ankle, 10, (255, 0, 255), -1)
    cv2.circle(img, ankle, 10, (255, 255, 255), 2)
    
    # Knee (larger, with angle)
    cv2.circle(img, knee, 18, color, -1)
    cv2.circle(img, knee, 18, (255, 255, 255), 2)
    
    # --- ANGLE TEXT ON KNEE ---
    angle_text = str(int(angle))
    text_size, _ = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_DUPLEX, 0.7, 2)
    
    # Position: right of knee
    text_x = knee[0] + 25
    text_y = knee[1] + 5
    
    # Background box for readability
    padding = 5
    cv2.rectangle(img, 
                  (text_x - padding, text_y - text_size[1] - padding),
                  (text_x + text_size[0] + padding, text_y + padding),
                  (0, 0, 0), -1)
    
    # Angle text
    cv2.putText(img, angle_text, (text_x, text_y),
                cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 2)


def draw_waiting_screen(img):
    """
    Draws the 'waiting for user' overlay.
    """
    h, w = img.shape[:2]
    
    # Dark overlay
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, 0.7, img, 0.3, 0)
    
    # Message
    text = "PLEASE STEP INTO FRAME"
    text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)
    text_x = (w - text_size[0]) // 2
    text_y = h // 2
    
    cv2.putText(img, text, (text_x, text_y),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
    
    # Sub-message
    sub_text = "Stand sideways to the camera"
    sub_size, _ = cv2.getTextSize(sub_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 1)
    sub_x = (w - sub_size[0]) // 2
    
    cv2.putText(img, sub_text, (sub_x, text_y + 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (150, 150, 150), 1)
    
    return img


# ==================== MAIN LOOP ====================

def main():
    global current_phase, total_reps, total_score, rep_error_flag, current_errors
    global front_leg, is_leg_locked, last_rep_time, missing_frames_count
    global last_valid_ui_data, min_angle_this_rep
    
    # Initialize camera
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    
    prev_time = time.time()
    
    print("\nControls:")
    print("  [Q] - Quit")
    print("  [R] - Reset stats")
    print("  [+] - Make easier")
    print("  [-] - Make harder")
    print("\nStarting...\n")

    while cap.isOpened():
        success, img = cap.read()
        if not success:
            print("Failed to read from camera")
            break
        
        # Mirror image for natural movement
        img = cv2.flip(img, 1)
        h, w = img.shape[:2]
        
        # --- 1. RUN YOLO INFERENCE ---
        results = model(img, verbose=False, conf=DETECTION_CONFIDENCE)[0]
        
        person_detected = False
        kp = None
        
        if results.keypoints and len(results.keypoints.data) > 0:
            kp = results.keypoints.data[0].cpu().numpy()
            
            # Check if both ankles are visible with good confidence
            left_ankle_conf = kp[15][2]
            right_ankle_conf = kp[16][2]
            
            if left_ankle_conf > 0.5 and right_ankle_conf > 0.5:
                person_detected = True
                missing_frames_count = 0
            else:
                missing_frames_count += 1
        else:
            missing_frames_count += 1
        
        # --- 2. PROCESS MOVEMENT ---
        if person_detected:
            
            # A. DETECT FRONT LEG
            # Only update when standing (not locked during movement)
            detected_leg = detect_front_leg(kp)
            
            if not is_leg_locked:
                front_leg = detected_leg
            
            # B. CALCULATE KNEE ANGLE
            idx = KP_MAP[front_leg]
            hip = kp[idx["hip"]][:2]
            knee = kp[idx["knee"]][:2]
            ankle = kp[idx["ankle"]][:2]
            
            raw_angle = calc_angle_3pts(hip, knee, ankle)
            
            # Smooth the angle
            angle_history.append(raw_angle)
            smooth_angle = sum(angle_history) / len(angle_history)
            
            # C. LEG LOCK LOGIC (Hysteresis)
            # Lock the leg when user starts going down
            # Unlock when fully standing
            if smooth_angle < (ANGLE_UP_THRESHOLD - 10):
                is_leg_locked = True
            elif smooth_angle > (ANGLE_UP_THRESHOLD - 5):
                is_leg_locked = False
            
            # D. TRACK MINIMUM ANGLE THIS REP
            if current_phase == "DOWN":
                min_angle_this_rep = min(min_angle_this_rep, smooth_angle)
            
            # E. CHECK FORM
            current_errors = check_form(kp, front_leg, smooth_angle, 
                                        current_phase, min_angle_this_rep)
            
            # Flag if any error occurred during DOWN phase
            if current_errors and current_phase == "DOWN":
                rep_error_flag = 1
            
            # F. REP COUNTING STATE MACHINE
            # Transition: UP -> DOWN
            if current_phase == "UP" and smooth_angle < ANGLE_DOWN_THRESHOLD:
                current_phase = "DOWN"
                min_angle_this_rep = smooth_angle  # Start tracking min angle
            
            # Transition: DOWN -> UP (count rep)
            if current_phase == "DOWN" and smooth_angle > ANGLE_UP_THRESHOLD:
                current_time = time.time()
                
                # Debounce: prevent double counting
                if current_time - last_rep_time > 0.8:
                    total_reps += 1
                    
                    # Calculate score (100 if perfect, 70 if errors)
                    if rep_error_flag == 0:
                        score = 100
                    else:
                        score = 70
                    
                    total_score += score
                    
                    # Print rep info
                    status = "PERFECT" if score == 100 else "FORM ERROR"
                    print(f"Rep {total_reps}: {score} pts ({status})")
                    
                    # Reset for next rep
                    current_phase = "UP"
                    rep_error_flag = 0
                    min_angle_this_rep = 999
                    last_rep_time = current_time

            # G. DRAW SKELETON
            draw_skeleton(img, kp, front_leg, smooth_angle)
            
            # H. UPDATE UI DATA
            last_valid_ui_data = {
                "reps": total_reps,
                "score": total_score,
                "phase": current_phase,
                "leg": front_leg,
                "angle": smooth_angle,
                "errors": current_errors
            }

        # --- 3. CALCULATE FPS ---
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time + 0.001)
        prev_time = curr_time
        
        # --- 4. DRAW UI ---
        if missing_frames_count < MAX_MISSING_FRAMES:
            draw_ui(img, last_valid_ui_data, fps)
        else:
            img = draw_waiting_screen(img)

        # --- 5. DISPLAY ---
        cv2.imshow("Bulgarian Split Squat - AI Trainer", img)
        
        # --- 6. HANDLE INPUT ---
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('r'):
            # Reset all stats
            total_reps = 0
            total_score = 0
            rep_error_flag = 0
            min_angle_this_rep = 999
            angle_history.clear()
            current_errors = []
            current_phase = "UP"
            
            last_valid_ui_data = {
                "reps": 0, "score": 0, "phase": "UP",
                "leg": front_leg, "angle": 180, "errors": []
            }
            
            print("\n--- STATS RESET ---\n")
        
        elif key == ord('+') or key == ord('='):
            update_tolerance(2)
        
        elif key == ord('-') or key == ord('_'):
            update_tolerance(-2)

    # --- CLEANUP ---
    cap.release()
    cv2.destroyAllWindows()
    
    # --- FINAL STATS ---
    print("\n" + "=" * 50)
    print("   WORKOUT COMPLETE!")
    print("=" * 50)
    print(f"   Total Reps: {total_reps}")
    
    if total_reps > 0:
        avg_score = total_score // total_reps
        print(f"   Total Score: {total_score}")
        print(f"   Average Score: {avg_score}")
        
        if avg_score >= 95:
            print("\n   EXCELLENT FORM!")
        elif avg_score >= 80:
            print("\n   GREAT JOB!")
        else:
            print("\n   KEEP PRACTICING!")
    
    print("=" * 50)


# ==================== ENTRY POINT ====================

if __name__ == "__main__":
    main()

   BULGARIAN SPLIT SQUAT - AI TRAINER
Loading YOLOv11 Model...
Model loaded successfully!

Controls:
  [Q] - Quit
  [R] - Reset stats
  [+] - Make easier
  [-] - Make harder

Starting...

Rep 1: 70 pts (FORM ERROR)
Rep 2: 70 pts (FORM ERROR)
Rep 3: 70 pts (FORM ERROR)
Rep 4: 70 pts (FORM ERROR)
Rep 5: 70 pts (FORM ERROR)
Rep 6: 70 pts (FORM ERROR)
Rep 7: 70 pts (FORM ERROR)
Rep 8: 70 pts (FORM ERROR)
Rep 9: 70 pts (FORM ERROR)
Rep 10: 70 pts (FORM ERROR)
Rep 11: 70 pts (FORM ERROR)
Rep 12: 70 pts (FORM ERROR)

   WORKOUT COMPLETE!
   Total Reps: 12
   Total Score: 840
   Average Score: 70

   KEEP PRACTICING!
